In [1]:
!pip install torcheval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 5.3 MB/s eta 0:00:00


In [31]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
# from ignite.metrics import PSNR
from torcheval.metrics import PeakSignalNoiseRatio
import torch.nn.functional as F

In [32]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image

In [33]:
# ignoring the warning messages
import warnings
from IPython.display import display
warnings.filterwarnings('ignore')

In [34]:
class TrainDataset(Dataset):
    def __init__(self, image_path, gt_path, transform = None):
        self.image_path = image_path
        self.gt_path = gt_path
        self.transform = transform
        self.image_list = os.listdir(image_path)

    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self, idx):

        img_name = self.image_list[idx]
        img, number = img_name.split('_')
        val_img = f'gt_{number}'

        noise_img = Image.open(os.path.join(self.image_path, img_name)).convert('RGB')

        gt_img = Image.open(os.path.join(self.gt_path, val_img)).convert('RGB')

        if self.transform:
            noise_img = self.transform(noise_img)
            gt_img = self.transform(gt_img)

        return noise_img, gt_img

In [35]:
class TestDataset(Dataset):
    def __init__(self, image_path, transform = None):
        self.image_path = image_path
        self.transform = transform
        self.image_list = os.listdir(image_path)

    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self, idx):

        img_name = self.image_list[idx]

        noise_img = Image.open(os.path.join(self.image_path, img_name))

        if self.transform:
            noise_img = self.transform(noise_img)

        return noise_img, img_name

In [36]:
noise_img_transform = transforms.Compose([
    # transforms.Resize((256, 256)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [37]:
noise_train_path = r'/kaggle/input/dlp-jan-2025-nppe-3/archive/train/train'
gt_train_path = r'/kaggle/input/dlp-jan-2025-nppe-3/archive/train/gt'

noise_val_path = r'/kaggle/input/dlp-jan-2025-nppe-3/archive/val/val'
gt_val_path = r'/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt'

noise_test_path = r'/kaggle/input/dlp-jan-2025-nppe-3/archive/test'

In [38]:
batch_size = 16

In [39]:
train_dataset = TrainDataset(noise_train_path, gt_train_path, transform = noise_img_transform)

val_dataset = TrainDataset(noise_val_path, gt_val_path, transform = noise_img_transform)

test_dataset = TestDataset(noise_test_path, transform = noise_img_transform)


In [40]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [41]:
class Conv2dBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, activation=True):
        super(Conv2dBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.norm = nn.BatchNorm2d(out_channels)
        self.activation = activation

    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        if self.activation:
            x = F.leaky_relu(x, 0.2)
        return x

In [42]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            Conv2dBlock(in_channels, in_channels),
            Conv2dBlock(in_channels, in_channels, activation=False)
        )

    def forward(self, x):
        return x + self.block(x)

In [43]:
class AttentionBlock(nn.Module):
    def __init__(self, in_channels):
        super(AttentionBlock, self).__init__()
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, in_channels // 16, 1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels // 16, in_channels, 1, padding=0),
            nn.Sigmoid()
        )

    def forward(self, x):
        attention = self.channel_attention(x)
        return x * attention

In [44]:
class UpsampleBlock(nn.Module):
    def __init__(self, in_channels, out_channels, scale_factor=2):
        super(UpsampleBlock, self).__init__()
        self.upsample = nn.Upsample(scale_factor=scale_factor, mode='bilinear', align_corners=False)
        self.conv = Conv2dBlock(in_channels, out_channels)

    def forward(self, x):
        x = self.upsample(x)
        x = self.conv(x)
        return x

In [45]:
class DenoiseSuperResNet(nn.Module):
    def __init__(self):
        super(DenoiseSuperResNet, self).__init__()
        self.initial = Conv2dBlock(3, 64)

        # Encoder
        self.enc1 = Conv2dBlock(64, 128, stride=2)
        self.enc2 = Conv2dBlock(128, 256, stride=2)
        self.enc3 = Conv2dBlock(256, 512, stride=2)

        # Bottleneck
        self.bottleneck = nn.Sequential(
            *[ResidualBlock(512) for _ in range(4)],
            AttentionBlock(512)
        )

        # Decoder
        self.dec3 = UpsampleBlock(512, 256)
        self.dec2 = UpsampleBlock(256, 128)
        self.dec1 = UpsampleBlock(128, 64)

        # Residual refinement
        self.residual = nn.Sequential(*[ResidualBlock(64) for _ in range(4)])

        # Final upsampling
        self.upsample1 = UpsampleBlock(64, 64, scale_factor=2)
        self.upsample2 = UpsampleBlock(64, 64, scale_factor=2)

        # Output layer
        self.output = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        # Initial feature extraction
        x = self.initial(x)

        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(enc1)
        enc3 = self.enc3(enc2)

        # Bottleneck
        x = self.bottleneck(enc3)

        # Decoder with skip connections
        x = self.dec3(x + enc3)
        x = self.dec2(x + enc2)
        x = self.dec1(x + enc1)

        # Residual refinement and upsampling
        x = self.residual(x)
        x = self.upsample1(x)
        x = self.upsample2(x)

        # Output
        return self.output(x)

In [46]:
def training(model, criterion, psnr_metric, optimizer, train_loader, epoch, epochs, device, scheduler = None):

    model.train()
    running_loss = 0.0
    psnr_metric.reset()

    with tqdm(total = len(train_loader), desc=f'Epoch {epoch+1}/{epochs}', unit='batch') as tepoch:
        for noise_imgs, gt_imgs in train_loader:
            noise_imgs, gt_imgs = noise_imgs.to(device), gt_imgs.to(device)

            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                
                outputs = model(noise_imgs)
                loss = criterion(outputs, gt_imgs)
            
            scaler.scale(loss).backward()

            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            # running_loss += total_loss.item()
            psnr_value = psnr_metric.update(outputs, gt_imgs)
            psnr_ = psnr_value.compute().item()

            tepoch.set_postfix({'Training MSE': running_loss / len(train_loader), 'Training PSNR':psnr_})
            tepoch.update(1)
        if scheduler:
            scheduler.step()

    return running_loss/len(train_loader)

In [47]:
def validation(model, criterion,psnr_metric, val_loader, epoch, epochs, device):

    model.eval()
    running_loss = 0.0
    psnr_metric.reset()

    with torch.no_grad():
        with tqdm(total=len(val_loader), desc=f'Epoch {epoch + 1}/{epochs}', unit='batch') as tepoch:

            for noise_imgs, gt_imgs in val_loader:
                noise_imgs, gt_imgs = noise_imgs.to(device), gt_imgs.to(device)

                # Forward pass
                outputs = model(noise_imgs)
                loss = criterion(outputs, gt_imgs)


                running_loss += loss.item()
                # running_loss += total_loss.item()
                psnr_value = psnr_metric.update(outputs, gt_imgs)
                psnr_ = psnr_value.compute().item()

                tepoch.set_postfix({'Validation MSE': running_loss / len(val_loader), 'Validation PSNR': psnr_})
                tepoch.update(1)

                

    return running_loss / len(val_loader)


In [48]:
def train_model(model, criterion,psnr_metric, optimizer, train_loader, val_loader, num_epochs, device, scheduler = None):

    best_val_loss = float('inf')

    for epoch in range(num_epochs):

        train_loss = training(model, criterion,psnr_metric, optimizer, train_loader, epoch, num_epochs, device, scheduler)

        val_loss = validation(model, criterion, psnr_metric, val_loader, epoch, num_epochs, device)
        # scheduler.step(val_loss)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'best_model_{epoch+1}.pth')
            print(f'Model is saved saved with the MSE Loss {best_val_loss}')
            
        # psnr_metric.reset()

        torch.cuda.empty_cache()
            
    return model 

In [49]:
model = DenoiseSuperResNet()

In [50]:
# shift the model to the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model = nn.DataParallel(model)  # Use GPUs 0 and 1
model = model.to(device)

In [51]:
# model.load_state_dict(torch.load('/kaggle/input/13_6983_model_37_new/pytorch/default/1/best_model_37.pth'))

In [52]:
num_epochs = 150
criterion = nn.MSELoss()
psnr_metric = PeakSignalNoiseRatio().to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6, weight_decay=1e-7)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.3, verbose=True)
scaler = torch.cuda.amp.GradScaler()

In [53]:
model = train_model(model, criterion,psnr_metric, optimizer, train_loader, val_loader, num_epochs, device)

Epoch 1/150: 100%|██████████| 17/17 [00:14<00:00,  1.16batch/s, Validation MSE=0.159, Validation PSNR=-2.54] 


Model is saved saved with the MSE Loss 0.15930018179556904


Epoch 2/150: 100%|██████████| 17/17 [00:14<00:00,  1.19batch/s, Validation MSE=0.124, Validation PSNR=-1.46] 


Model is saved saved with the MSE Loss 0.12442299723625183


Epoch 3/150: 100%|██████████| 17/17 [00:14<00:00,  1.19batch/s, Validation MSE=0.0943, Validation PSNR=-0.261]


...

Epoch 143/150: 100%|██████████| 17/17 [00:14<00:00,  1.21batch/s, Validation MSE=0.000344, Validation PSNR=24.1]


Model is saved saved with the MSE Loss 0.00034444285762112807


Epoch 148/150: 100%|██████████| 17/17 [00:14<00:00,  1.21batch/s, Validation MSE=0.00034, Validation PSNR=24.2] 


Model is saved saved with the MSE Loss 0.0003398933485561215


Epoch 150/150: 100%|██████████| 17/17 [00:14<00:00,  1.21batch/s, Validation MSE=0.000347, Validation PSNR=24.1]


In [54]:
def test_prediction(model, test_loader, device, path):
    model.eval()
    with torch.no_grad():
        with tqdm(total = len(test_loader), desc = 'Testing', unit = 'batch') as tepoch:
            for noise_imgs, img_name in test_loader:
                noise_imgs = noise_imgs.to(device)

                outputs = model(noise_imgs)

                for idx in range(outputs.shape[0]):
                    # Scale the output tensor to [0, 1]
                    predicted_img = outputs[idx].squeeze(0).cpu()
                    out_path = os.path.join(path, f"{img_name[idx]}")
                    save_image(predicted_img, out_path)

                tepoch.update(1)

In [55]:
# model.load_state_dict(torch.load('/kaggle/working/best_model_51.pth'))

In [56]:
os.makedirs('/kaggle/working/outputs/predicted_images', exist_ok=True)
path = '/kaggle/working/outputs/predicted_images/'

In [57]:
test_prediction(model, test_loader, device, path)

Testing: 100%|██████████| 4/4 [00:18<00:00,  4.62s/batch]


In [58]:
import os
import numpy as np
import pandas as pd
from PIL import Image

def images_to_csv(folder_path, output_csv):
    data_rows = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('L') 
            image_array = np.array(image).flatten()[::8]
            # Replace 'test_' with 'gt_' in the ID
            image_id = filename.split('.')[0].replace('test_', 'gt_')
            data_rows.append([image_id, *image_array])
    column_names = ['ID'] + [f'pixel_{i}' for i in range(len(data_rows[0]) - 1)]
    df = pd.DataFrame(data_rows, columns=column_names)
    df.to_csv(output_csv, index=False)
    print(f'Successfully saved to {output_csv}')

folder_path = '/kaggle/working/outputs/predicted_images'
output_csv = 'submission_202_last_model.csv'
images_to_csv(folder_path, output_csv)

Successfully saved to submission_202_last_model.csv
